In [ ]:
import numpy as np
import keras.utils as image
from PIL import Image
import skimage.measure
import math
import sys
import os
import time

from keras import backend as K
import tensorflow as tf

2024-05-01 15:43:41.190403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 15:43:42.853100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/chiu/.local/lib/python3.10/site_packages/tensorrt:/usr/local/cuda-12.4/targets/x86_64-linux/lib:/usr/local/cuda-12.4/lib64:/usr/local/cuda-12.0/targets/x86_64-linux/lib:
2024-05-01 15:43:42.853179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file

In [ ]:
filename = 'sheepdog'
img = image.load_img('./SALICON/'+filename+'.jpg')
height,width = img.size
channels = 3
img = image.img_to_array(img)
img = img[:, :, ::-1]
img = np.expand_dims(img, axis=0)
tex_pixels = tf.squeeze(img)/255.

In [ ]:
threshold=0.1
max_sigma = 235
sigma_sequence = [1.0, 1.5, 2.0, 4.0, 7.0, 10.0]
increment = 5.0
increment_steps = [5.0, 10.0, 20.0, 40.0, 50.0]
increment_index = 0
step_counter = 0
current_sigma = sigma_sequence[-1] + increment
while current_sigma <= max_sigma:
    sigma_sequence.append(current_sigma)
    step_counter += 1
    if step_counter % 5 == 0 and increment_index < len(increment_steps) - 1:
        increment_index += 1
        increment = increment_steps[increment_index]
    current_sigma += increment
    if current_sigma >= max_sigma:
        sigma_sequence.append(max_sigma)
        break

In [ ]:
tsg_lists = {}
for sigma in sigma_sequence:
    kernel_size = int(4 * sigma - 1)
    current_height = int(kernel_size / 2)
    current_width = int(kernel_size / 2)
    tsg_current = np.array([[np.exp(-((i_ref_p)**2 + (j_ref_p)**2) / (2*sigma**2)) \
                          for j_ref_p in range(-current_width, current_width + 1)] \
                         for i_ref_p in range(-current_height, current_height + 1)])

    tsg_current = tsg_current / np.sum(tsg_current)
    tsg_current = np.expand_dims(tsg_current, axis=-1)  # Add a third dimension for 'in_channels'
    tsg_current = np.expand_dims(tsg_current, axis=-1)  # Add a fourth dimension for 'out_channels'
    tsg_current = np.repeat(tsg_current, 3, axis=-1)
    tsg_current_tf = tf.convert_to_tensor(tsg_current, dtype=tf.float32)
    tsg_lists[sigma] = tsg_current_tf

In [ ]:
mean_tex_all = []
var_tex_all = []
for sigma in sigma_sequence:
    tsg = tsg_lists[sigma]
    tex = tf.expand_dims(tex_pixels, axis=0)  # Add batch dimension
    kernel_size = len(tsg)
    padding_size = int((kernel_size + 1) / 2) - 1
    padding = tf.constant([[0, 0], [padding_size, padding_size], [padding_size, padding_size], [0, 0]])
    tex_padded = tf.pad(tex, padding, "SYMMETRIC")
    # Convolution
    mean_tex = tf.nn.conv2d(tex_padded, tsg, strides=[1, 1, 1, 1], padding='VALID')
    mean_tex_padded = tf.pad(mean_tex, padding, "SYMMETRIC")
    squared_diff_tex = (tex_padded - mean_tex_padded) ** 2
    var_tex = tf.maximum(tf.nn.conv2d(squared_diff_tex, tsg, strides=[1, 1, 1, 1], padding='VALID'), 0)
    mean_tex_all.append(mean_tex)
    var_tex_all.append(var_tex)

mean_tex_all = np.concatenate(mean_tex_all,axis=0)
var_tex_all = np.concatenate(var_tex_all,axis=0)
wd_sigma_list = (mean_tex_all[1:,:,:,:] - mean_tex_all[:-1,:,:,:])**2 + var_tex_all[1:,:,:,:] + var_tex_all[:-1,:,:,:] -\
                                            2*tf.math.sqrt(var_tex_all[1:,:,:,:]*var_tex_all[:-1,:,:,:])
wd_sigma_list = tf.math.reduce_mean(wd_sigma_list,axis=3)

In [ ]:
len(sigma_sequence)

23

In [ ]:
wd_sigma_list

<tf.Tensor: shape=(22, 480, 480), dtype=float32, numpy=
array([[[3.0978464e-07, 2.7119140e-07, 9.9703016e-08, ...,
         4.2687549e-08, 2.5484357e-07, 3.3015252e-07],
        [3.0978464e-07, 2.7119140e-07, 9.9703016e-08, ...,
         1.3677284e-08, 1.2293613e-07, 2.5500006e-07],
        [3.0978464e-07, 2.7119140e-07, 9.9703016e-08, ...,
         7.8977491e-09, 1.3684598e-08, 4.2737952e-08],
        ...,
        [5.8261314e-05, 1.0739290e-04, 1.0370372e-04, ...,
         7.6804346e-05, 3.9270653e-05, 9.7602395e-05],
        [3.8807699e-05, 6.5753047e-05, 6.3208478e-05, ...,
         3.3165968e-05, 2.0603688e-05, 9.7295000e-05],
        [3.8805956e-05, 1.0643967e-04, 8.4387953e-05, ...,
         1.8008228e-05, 3.3787801e-06, 1.2425105e-05]],

       [[2.4247140e-07, 1.3556784e-07, 2.0695704e-07, ...,
         1.6173999e-08, 1.6429491e-08, 8.5801616e-08],
        [2.4247140e-07, 1.3556784e-07, 2.0695704e-07, ...,
         8.9912646e-09, 3.3783181e-09, 1.6431613e-08],
        [2.744230

In [ ]:
sigma_map = np.zeros([height, width])

for x in range(height):
    for y in range(width):
        # print(f"Processing position: ({x}, {y})")
        pixel_wd_values = wd_sigma_list[:,x,y].numpy().tolist()
        wd_values_under_5 = wd_sigma_list[:4,x,y].numpy().tolist()

        # print("Pixel Value:", pixel_wd_values)
        max_wd_under_5 = max(wd_values_under_5) if wd_values_under_5 else 0
        # print("Max value under sigma 5:", max_wd_under_5)

        # Check if max_wd_under_5 is among the top 5 values in the full list
        top_values = sorted(pixel_wd_values, reverse=True)[:5]
        is_top_five = max_wd_under_5 in top_values

        # Check if max_wd_under_5 is the overall maximum
        overall_max = max(pixel_wd_values)
        is_overall_max = max_wd_under_5 == overall_max

        differences = np.diff(pixel_wd_values)
        # print("differences:", differences)

        # Find the first index where sigma is greater than 10
        if is_top_five:
            start_index = pixel_wd_values.index(max_wd_under_5) + 1
        else:
            start_index = next(
                (index-1 for index, sigma in enumerate(sigma_sequence) if sigma >= 15),
                len(sigma_sequence) - 1
            )

        trend_changes = []

        i = 0
        while i < len(differences):
            if differences[i] > 0:
                start = i
                while i < len(differences) and differences[i] > 0:
                    i += 1
                trend_changes.append((start, i-1))
            i += 1
        # print("Trend_change:", trend_changes)

        if is_overall_max:
            found_significant = False
            index = pixel_wd_values.index(overall_max) + 1
            for i in range(index, len(sigma_sequence) - 1):
                if pixel_wd_values[i] * 10 > max_wd_under_5:
                    sigma_map[x, y] = sigma_sequence[i]
                    found_significant = True
                    # print(f"Optimal Sigma value at ({x}, {y}): {sigma_map[x, y]}")
                    break
            if not found_significant:
                sigma_map[x, y] = 0
                # print(f"No significant sigma found at ({x}, {y}), set to 0")
            continue

        for start, end in trend_changes:
            if start >= start_index or (start <= start_index <= end):
                if pixel_wd_values[end] / max_wd_under_5 > threshold:
                    sigma_map[x, y] = sigma_sequence[end + 1]
                    # print(f"Optimal Sigma value at ({x}, {y}): {sigma_map[x, y]}")
                    break

In [ ]:
sigma_map.shape

(480, 480)

In [ ]:
np.save(filename+'_sigma_map_v6.npy', sigma_map)
sigma_map_normalized = (sigma_map * 255 / np.max(sigma_map)).astype(np.uint8)
im = Image.fromarray(sigma_map_normalized, mode='L')
fname = filename + '_sigma_map_heatmap.png'
im.save(fname)
print('Image saved as', fname)

Image saved as sheepdog_sigma_map_heatmap.png
